# Load Packages

In [1]:
# --- System and file handling ---
import os
import pandas as pd
import imageio

# --- Numerical and array operations ---
import numpy as np

# --- Raster handling ---
import rasterio
from rasterio.plot import show
from rasterio.windows import from_bounds

# --- Geo-spatial handling ---
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# --- Plotting ---
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import cm

In [2]:
rain = pd.read_csv('rain_data/arm-rainfall-adm2-full-1.csv')
rain.head()

,date,adm2_id,ADM2_PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
0,#date,#adm2+id,#adm2+code,#indicator+n_pixels+num,#indicator+rfh+num,#indicator+rfh_avg+num,#indicator+r1h+num,#indicator+r1h_avg+num,#indicator+r3h+num,#indicator+r3h_avg+num,#indicator+rfq+pct,#indicator+r1q+pct,#indicator+r3q+pct,#status
1,1981-01-01,1009196,AM064,27.0,3.5185,9.4914,NaN,NaN,NaN,NaN,58.7834,NaN,NaN,final
2,1981-01-11,1009196,AM064,27.0,7.037,7.742,NaN,NaN,NaN,NaN,94.4676,NaN,NaN,final
3,1981-01-21,1009196,AM064,27.0,6.1481,11.7074,16.7037,28.9407,NaN,NaN,66.7258,68.5752,NaN,final
4,1981-02-01,1009196,AM064,27.0,4.7778,8.7333,17.963,28.1827,NaN,NaN,71.1974,73.2346,NaN,final


In [6]:
rain_work = rain.copy()
# remove any non-data/header rows (e.g. the "#date" header row present in the dataframe)
rain_work = rain_work[~rain_work['date'].astype(str).str.startswith('#')].copy()

# parse dates robustly (invalid parses become NaT), then drop rows without valid dates
rain_work['date'] = pd.to_datetime(rain_work['date'], errors='coerce', infer_datetime_format=True)

rain_work = rain_work.sort_values('date')
rain_work.head()

C:\Users\jonas\AppData\Local\Temp\ipykernel_13132\1364805953.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rain_work['date'] = pd.to_datetime(rain_work['date'], errors='coerce', infer_datetime_format=True)


,date,adm2_id,ADM2_PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
1,1981-01-01,1009196,AM064,27.0,3.5185,9.4914,NaN,NaN,NaN,NaN,58.7834,NaN,NaN,final
15321,1981-01-01,1009225,AM114,15.0,2.6,8.6356,NaN,NaN,NaN,NaN,55.7366,NaN,NaN,final
10725,1981-01-01,1009222,AM043,13.0,2.9231,7.4564,NaN,NaN,NaN,NaN,63.6064,NaN,NaN,final
39833,1981-01-01,1009203,AM072,40.0,2.0,6.0583,NaN,NaN,NaN,NaN,63.3007,NaN,NaN,final
22981,1981-01-01,1009230,AM111,43.0,3.1163,11.6372,NaN,NaN,NaN,NaN,48.7839,NaN,NaN,final
